# Playwright를 활용한 Web Scrapping 실습
- Jupyter Notebook 환경에서는, playwright를 async(비동기 방식으로 사용해야 합니다.)
- Jupyter에서는 이미 Event Loop가 실행되고 있어, sync 방식으로 수행하면 오류가 발생합니다. (기존에 있던 루프와의 충돌)

In [ ]:
playwright 패키지의 playwright.async_api를 활용합니다.
이 패키지의 함수를 활용할 때는 await 명령어를 앞에 붙여주어야 합니다.


| id | 함수                                  | 설명                      | 주요 사용 상황               |
| -- | ----------------------------------- | ----------------------- | ---------------------- |
| 1  | `page.goto(url)`                    | 지정한 URL로 이동             | 웹페이지 접속                |
| 2  | `page.wait_for_selector(selector)`  | 특정 요소가 나타날 때까지 대기       | 로딩 대기                  |
| 3  | `locator(selector)`                 | 요소를 찾는 객체 생성            | 후속 조작 준비               |
| 4  | `locator.click()`                   | 요소 클릭                   | 버튼/링크 클릭               |
| 5  | `locator.fill(value)`               | 입력 필드 값 채우기             | 검색창, 로그인 입력            |
| 6  | `locator.text_content()`            | 요소의 텍스트 가져오기            | 기사 제목, 텍스트 추출          |
| 7  | `locator.get_attribute(name)`       | 요소 속성 가져오기              | 링크(`href`), 이미지(`src`) |
| 8  | `page.query_selector(selector)`     | CSS 셀렉터로 **첫 번째 요소** 찾기 | 단일 요소 필요할 때            |
| 9  | `page.query_selector_all(selector)` | CSS 셀렉터로 **모든 요소** 찾기   | 목록/리스트 추출              |
| 10 | `page.screenshot(path=...)`         | 페이지 스크린샷 저장             | 디버깅, 결과 기록             |
| 11 | `locator.inner_text()`   | 요소 내부의 표시 텍스트 가져오기 (렌더링된 값 기준) | str |

```
async with async_playwright() as p:
```
- 비동기 환경에서 Playwright를 사용하기 위한 필수적인 문법

- 위 코드를 통해, async_playwright()를 호출하여 Playwright 라이브러리를 초기화하고, p라는 변수에 할당합니다.

- with 블록을 벗어나면, Playwright는 자동으로 모든 브라우저 인스턴스를 닫고 관련 리소스를 정리합니다. 이는 사용자가 직접 browser.close()를 호출하지 않아도 되므로 코드의 안정성을 높이고 자원 누수를 방지합니다.

In [11]:
import asyncio
from playwright.async_api import async_playwright
async with async_playwright() as p:
    # 1. 브라우저 실행 (모든 동작 앞에 await 키워드를 붙입니다)
    browser = await p.chromium.launch(headless=False)  # headless=True로 설정하면 브라우저를 안 띄웁니다.
        
    # 2. 새 페이지 열기
    page = await browser.new_page()
        
    # 3. 원하는 URL로 이동
    await page.goto("https://econotimes.com/search?v=tesla&search=")
        
    # 4. 페이지 제목 가져와서 출력하기
    page_title = await page.title()
    print(f"현재 페이지 제목: {page_title}")

현재 페이지 제목: Breaking News, Business, Financial and Economic News, World News from EconoTimes


In [ ]:
# 기사 제목의 xpath 패턴 파악  
//*[@id="archivePage"]/div/div[2]/div[1]/p[1]/a  
//*[@id="archivePage"]/div/div[2]/div[2]/p[1]/a  
//*[@id="archivePage"]/div/div[2]/div[3]/p[1]/a  

# 패턴이 보이시나요?  


# 아래와 같은 패턴을 보임 (i : 1~)
//*[@id="archivePage"]/div/div[2]/div[i]/p[1]/a

위와 같이 div[]의 숫자가 1씩 증가하는 패턴을 보입니다.
해당 패턴을 가지는 XPath 요소를 추출하고 싶을 때 대괄호를 생략하고, '일반적인' XPath를 정의할 수 있습니다.
또는, 반복문을 활용하여 진행해도 됩니다.

```
for i in range(1, self.max_articles + 1):
    try:
    xpath = f'//*[@id="archivePage"]/div/div[2]/div[{i}]/p[1]/a'
    link = await page.query_selector(f'xpath={xpath}')
    title = await link.inner_text()
    url = await link.get_attribute('href')

```

In [ ]:
general_xpath = '//*[@id="archivePage"]/div/div[2]/div/p[1]/a'
elements = await page.locator(f"xpath={general_xpath}").all()

# 15개 기사 추출

In [19]:
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    print("🚀 Tesla 기사 제목 추출 시작!")
    
    # 브라우저 실행
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    
    # Tesla 검색 페이지로 이동
    await page.goto("https://econotimes.com/search?v=tesla&search=")
    await asyncio.sleep(3)
    
    # XPath를 사용해서 모든 기사 제목 요소 찾기
    general_xpath = '//*[@id="archivePage"]/div/div[2]/div/p[1]/a'
    elements = await page.locator(f"xpath={general_xpath}").all()   # .all()을 쓰면 모든 요소를 가져옴.    
    
    print(f"\n📰 발견된 Tesla 기사: {len(elements)}개\n")
    print("=" * 80)
    
    # 각 기사 제목 추출하고 출력
    for i, element in enumerate(elements, 1):
        try:
            # 기사 제목 텍스트 추출
            title = await element.text_content()
            href = await element.get_attribute('href')
            if title and href:
                # 상대 URL을 절대 URL로 변환
                if href.startswith('/'):
                    full_url = f"https://econotimes.com{href}"  #href가 완전한 링크를 주는지 아닌지 확인해야 함.
                else:
                    full_url = href
            
            if title:
                # 제목 출력
                print(f"{i:2d}. {title.strip()}")
                print(f"    🔗 {full_url}")

            
        except Exception as e:
            print(f"{i:2d}. ❌ 추출 실패: {e}")
    
    print("=" * 80)
    print(f"✅ 총 {len(elements)}개의 기사 제목 추출 완료!")
    
    # 브라우저 종료
    await browser.close()

🚀 Tesla 기사 제목 추출 시작!

📰 발견된 Tesla 기사: 15개

 1. Tesla Ordered to Pay $243M in Florida Autopilot Crash Verdict
    🔗 https://econotimes.com/Tesla-Ordered-to-Pay-243M-in-Florida-Autopilot-Crash-Verdict-1717846
 2. Tesla Car Sales in France, Denmark, and Sweden Plunge in July Amid Ongoing European Decline
    🔗 https://econotimes.com/Tesla-Car-Sales-in-France-Denmark-and-Sweden-Plunge-in-July-Amid-Ongoing-European-Decline-1717783
 3. Tesla Secures $4.3B Battery Deal with LG Energy for U.S. Storage Projects
    🔗 https://econotimes.com/Tesla-Secures-43B-Battery-Deal-with-LG-Energy-for-US-Storage-Projects-1717488
 4. Samsung Shares Dip After Rally on $16.5B Tesla Chip Deal
    🔗 https://econotimes.com/Samsung-Shares-Dip-After-Rally-on-165B-Tesla-Chip-Deal-1717364
 5. Tesla Inks $16.5B Chip Deal with Samsung for Next-Gen AI6 Production in Texas
    🔗 https://econotimes.com/Tesla-Inks-165B-Chip-Deal-with-Samsung-for-Next-Gen-AI6-Production-in-Texas-1717282
 6. Tesla Inks $16.5B Chip Deal with 

### 100개의 뉴스 기사의 제목과 링크를 추출하여 csv 파일로 저장
- 뉴스 기사를 더 보려면 웹페이지에서 'more'버튼을 더 눌러야 한다.

In [ ]:
# more 버튼
//*[@id="archivePage"]/div/div[3]/span  
//*[@id="archivePage"]/div/div[3]/span  
//*[@id="archivePage"]/div/div[3]/span  

# more 버튼을 계속 눌러도 xpath는 변하지 않음.
# more 버튼을 누를 때마다 15개의 뉴스기사가 추가 표시되는 것 확인.

In [ ]:
# 클릭 버튼의 xpath는 아래와 같다.
# //*[@id="archivePage"]/div/div[3]/span
# 위 코드에서 'more'버튼을 클릭하는 코드를 추가해, 더 많은 뉴스 기사를 크롤링 해보겠습니다.

#### 클릭하는 코드  
```python
xpath = '//*[@id="archivePage"]/div/div[3]/span'
await page.locator(f"xpath={xpath}").click()
```

In [20]:
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    print("🚀 Tesla 기사 제목 추출 시작!")
    
    # 브라우저 실행
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    
    # Tesla 검색 페이지로 이동
    await page.goto("https://econotimes.com/search?v=tesla&search=")
    await asyncio.sleep(3)

    #more버튼을 6번 누르고 앞의 과정 진행
    xpath = '//*[@id="archivePage"]/div/div[3]/span'

    for i in range(6):
        await page.locator(f"xpath={xpath}").click()
        await asyncio.sleep(1.5)


    
    # XPath를 사용해서 모든 기사 제목 요소 찾기
    general_xpath = '//*[@id="archivePage"]/div/div[2]/div/p[1]/a'
    elements = await page.locator(f"xpath={general_xpath}").all()
    
    print(f"\n📰 발견된 Tesla 기사: {len(elements)}개\n")
    print("=" * 80)
    
    # 각 기사 제목 추출하고 출력
    for i, element in enumerate(elements, 1):
        try:
            # 기사 제목 텍스트 추출
            title = await element.text_content()
            href = await element.get_attribute('href')
            if title and href:
                # 상대 URL을 절대 URL로 변환
                if href.startswith('/'):
                    full_url = f"https://econotimes.com{href}"  #href가 
                else:
                    full_url = href
            
            if title:
                # 제목 출력
                print(f"{i:2d}. {title.strip()}")
                print(f"    🔗 {full_url}")

            
        except Exception as e:
            print(f"{i:2d}. ❌ 추출 실패: {e}")
    
    print("=" * 80)
    print(f"✅ 총 {len(elements)}개의 기사 제목 추출 완료!")
    
    # 브라우저 종료
    await browser.close()

🚀 Tesla 기사 제목 추출 시작!

📰 발견된 Tesla 기사: 105개

 1. Tesla Ordered to Pay $243M in Florida Autopilot Crash Verdict
    🔗 https://econotimes.com/Tesla-Ordered-to-Pay-243M-in-Florida-Autopilot-Crash-Verdict-1717846
 2. Tesla Car Sales in France, Denmark, and Sweden Plunge in July Amid Ongoing European Decline
    🔗 https://econotimes.com/Tesla-Car-Sales-in-France-Denmark-and-Sweden-Plunge-in-July-Amid-Ongoing-European-Decline-1717783
 3. Tesla Secures $4.3B Battery Deal with LG Energy for U.S. Storage Projects
    🔗 https://econotimes.com/Tesla-Secures-43B-Battery-Deal-with-LG-Energy-for-US-Storage-Projects-1717488
 4. Samsung Shares Dip After Rally on $16.5B Tesla Chip Deal
    🔗 https://econotimes.com/Samsung-Shares-Dip-After-Rally-on-165B-Tesla-Chip-Deal-1717364


C:\Users\hyukj\anaconda3\Lib\traceback.py:265: RuntimeWarning: coroutine 'get_selector' was never awaited
  def __init__(self, filename, lineno, name, *, lookup_line=True,
Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


 5. Tesla Inks $16.5B Chip Deal with Samsung for Next-Gen AI6 Production in Texas
    🔗 https://econotimes.com/Tesla-Inks-165B-Chip-Deal-with-Samsung-for-Next-Gen-AI6-Production-in-Texas-1717282
 6. Tesla Inks $16.5B Chip Deal with Samsung to Boost AI Chip Production
    🔗 https://econotimes.com/Tesla-Inks-165B-Chip-Deal-with-Samsung-to-Boost-AI-Chip-Production-1717253
 7. Tesla Prepares Robotaxi Launch in Bay Area with Safety Drivers Amid Regulatory Hurdles
    🔗 https://econotimes.com/Tesla-Prepares-Robotaxi-Launch-in-Bay-Area-with-Safety-Drivers-Amid-Regulatory-Hurdles-1717201
 8. Tesla to Launch Chauffeur-Style Service in Bay Area Amid Robotaxi Delays
    🔗 https://econotimes.com/Tesla-to-Launch-Chauffeur-Style-Service-in-Bay-Area-Amid-Robotaxi-Delays-1717164
 9. Tesla Receives Shareholder Proposals on Potential xAI Investment
    🔗 https://econotimes.com/Tesla-Receives-Shareholder-Proposals-on-Potential-xAI-Investment-1717159
10. Tesla Faces Revenue Slump Amid EV Credit Cuts, Musk

In [24]:
# 📊 DataFrame으로 Tesla 기사 데이터 저장
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
from datetime import datetime

async with async_playwright() as p:
    
    # 브라우저 실행
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    
    # Tesla 검색 페이지로 이동
    await page.goto("https://econotimes.com/search?v=tesla&search=")
    await asyncio.sleep(3)

    # More 버튼을 6번 누르고 앞의 과정 진행
    print(" More 버튼 6번 클릭 중...")
    xpath = '//*[@id="archivePage"]/div/div[3]/span'

    for i in range(6):
        try:
            await page.locator(f"xpath={xpath}").click()
            await asyncio.sleep(1.5)
            print(f"   {i+1}번째 More 버튼 클릭 완료")
        except:
            print(f"   {i+1}번째 More 버튼 클릭 실패 (더 이상 없을 수 있음)")
            break
    
    # XPath를 사용해서 모든 기사 제목 요소 찾기
    general_xpath = '//*[@id="archivePage"]/div/div[2]/div/p[1]/a'
    elements = await page.locator(f"xpath={general_xpath}").all()
    
    print(f"\n📰 발견된 Tesla 기사: {len(elements)}개")
    print("🔍 데이터 추출 중...")
    
    # 데이터를 저장할 리스트
    articles_data = []
    
    # 각 기사 제목과 링크 추출
    for i, element in enumerate(elements, 1):
        try:
            # 기사 제목 텍스트 추출
            title = await element.text_content()
            href = await element.get_attribute('href')
            
            if title and href:
                # 상대 URL을 절대 URL로 변환
                if href.startswith('/'):
                    full_url = f"https://econotimes.com{href}"
                else:
                    full_url = href
                
                # 데이터를 딕셔너리로 저장 (datetime은 참고용으로 확인해주시면 됩니다)
                articles_data.append({
                    'index': i,
                    'title': title.strip(),
                    'url': full_url,
                    'crawled_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
            
        except Exception as e:
            print(f"❌ {i}번째 기사 추출 실패: {e}")
    
    # DataFrame 생성
    df = pd.DataFrame(articles_data)
    
    # 결과 출력
    print("=" * 100)
    print(f" DataFrame 생성 완료! 총 {len(df)}개의 기사 데이터")
    print("=" * 100)
    
    # DataFrame 정보 출력
    print("\n DataFrame 정보:")
    print(f"   • 행 개수: {len(df)}")
    print(f"   • 열 개수: {len(df.columns)}")
    print(f"   • 컬럼: {list(df.columns)}")
        
    # 브라우저 종료
    await browser.close()
    
    # DataFrame을 전역 변수로 저장 (Jupyter에서 사용하기 위해)
    tesla_df = df
    
print(f"\n 'tesla_df' 변수에 {len(tesla_df)}개 기사 데이터가 저장되었습니다!")
print("csv파일 저장할 때 tesla_df.to_csv('tesla_articles.csv')")

 More 버튼 6번 클릭 중...
   1번째 More 버튼 클릭 완료
   2번째 More 버튼 클릭 완료
   3번째 More 버튼 클릭 완료
   4번째 More 버튼 클릭 완료
   5번째 More 버튼 클릭 완료
   6번째 More 버튼 클릭 완료

📰 발견된 Tesla 기사: 105개
🔍 데이터 추출 중...
 DataFrame 생성 완료! 총 105개의 기사 데이터

 DataFrame 정보:
   • 행 개수: 105
   • 열 개수: 4
   • 컬럼: ['index', 'title', 'url', 'crawled_at']

 'tesla_df' 변수에 105개 기사 데이터가 저장되었습니다!
csv파일 저장할 때 tesla_df.to_csv('tesla_articles.csv')


In [23]:
tesla_df.head()

,index,title,url,crawled_at
0,1,Tesla Ordered to Pay $243M in Florida Autopilo...,https://econotimes.com/Tesla-Ordered-to-Pay-24...,2025-09-08 19:14:19
1,2,"Tesla Car Sales in France, Denmark, and Sweden...",https://econotimes.com/Tesla-Car-Sales-in-Fran...,2025-09-08 19:14:19
2,3,Tesla Secures $4.3B Battery Deal with LG Energ...,https://econotimes.com/Tesla-Secures-43B-Batte...,2025-09-08 19:14:19
3,4,Samsung Shares Dip After Rally on $16.5B Tesla...,https://econotimes.com/Samsung-Shares-Dip-Afte...,2025-09-08 19:14:19
4,5,Tesla Inks $16.5B Chip Deal with Samsung for N...,https://econotimes.com/Tesla-Inks-165B-Chip-De...,2025-09-08 19:14:19


# 뉴스 기사 본문 추출
- 우선 하나의 URL로부터 본문 추출

### copy selector 활용

In [ ]:
# 본문 영역에 F12 개발자 도구 클릭 - copy - copy selector
#view > div.articleViewWrap > div.articleContent > article

In [28]:
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    
    # 기사 페이지 이동
    await page.goto("https://econotimes.com/Tesla-Ordered-to-Pay-243M-in-Florida-Autopilot-Crash-Verdict-1717846")
    await asyncio.sleep(3)
    
    # 본문 추출 (copy selector)
    article_text = await page.locator("#view > div.articleViewWrap > div.articleContent > article").text_content()
    
    # 결과 출력
    print("기사 본문:")
    print("=" * 50)
    print(article_text[:300] + "...")  # 처음 300자만
    
    await browser.close()

기사 본문:

                    A Florida jury has ordered Tesla (NASDAQ: TSLA) to pay $243 million in damages over a 2019 fatal crash involving its Autopilot system, marking a rare legal defeat for the electric vehicle maker. The case centered on a Model S collision that killed Naibel Benavides Leon and sever...


### copy xpath 활용

In [30]:
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False)
    page = await browser.new_page()
    
    # 기사 페이지 이동
    await page.goto("https://econotimes.com/Tesla-Ordered-to-Pay-243M-in-Florida-Autopilot-Crash-Verdict-1717846")
    await asyncio.sleep(3)
    
    # XPath로 본문 추출
    xpath = '//*[@id="view"]/div[2]/div[3]/article'
    article_text = await page.locator(f"xpath={xpath}").text_content()
    
    # 결과 출력
    print("XPath로 추출한 기사 본문:")
    print("=" * 50)
    print(article_text)
    
    await browser.close()


XPath로 추출한 기사 본문:

                    A Florida jury has ordered Tesla (NASDAQ: TSLA) to pay $243 million in damages over a 2019 fatal crash involving its Autopilot system, marking a rare legal defeat for the electric vehicle maker. The case centered on a Model S collision that killed Naibel Benavides Leon and severely injured Dillon Angulo. Jurors in Miami federal court awarded $129 million in compensatory damages and $200 million in punitive damages, holding Tesla liable for 33% of the compensatory amount, or $42.6 million. The driver, George McGee, was found 67% liable but is not required to pay damages. The lawsuit alleged Tesla failed to limit Autopilot’s use to controlled-access highways, despite CEO Elon Musk’s public claims that the system drove better than humans. The crash occurred when McGee, distracted by a dropped cellphone, ran a stop sign and collided with the victims’ parked SUV. Tesla denied responsibility and vowed to appeal, arguing no vehicle in 2019—or today—could

### 본문을 포함한 dataframe 파일 만들기 (기사 10개)

In [47]:
urls=[]
for i in range(1,11):
    urls.append(tesla_df['url'][i])

In [48]:
urls

['https://econotimes.com/Tesla-Car-Sales-in-France-Denmark-and-Sweden-Plunge-in-July-Amid-Ongoing-European-Decline-1717783',
 'https://econotimes.com/Tesla-Secures-43B-Battery-Deal-with-LG-Energy-for-US-Storage-Projects-1717488',
 'https://econotimes.com/Samsung-Shares-Dip-After-Rally-on-165B-Tesla-Chip-Deal-1717364',
 'https://econotimes.com/Tesla-Inks-165B-Chip-Deal-with-Samsung-for-Next-Gen-AI6-Production-in-Texas-1717282',
 'https://econotimes.com/Tesla-Inks-165B-Chip-Deal-with-Samsung-to-Boost-AI-Chip-Production-1717253',
 'https://econotimes.com/Tesla-Prepares-Robotaxi-Launch-in-Bay-Area-with-Safety-Drivers-Amid-Regulatory-Hurdles-1717201',
 'https://econotimes.com/Tesla-to-Launch-Chauffeur-Style-Service-in-Bay-Area-Amid-Robotaxi-Delays-1717164',
 'https://econotimes.com/Tesla-Receives-Shareholder-Proposals-on-Potential-xAI-Investment-1717159',
 'https://econotimes.com/Tesla-Faces-Revenue-Slump-Amid-EV-Credit-Cuts-Musk-Bets-on-Robotaxi-Future-1716928',
 'https://econotimes.com/US

In [49]:
article_content = []
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=False)
    page = await browser.new_page()
    xpath = '//*[@id="view"]/div[2]/div[3]/article'
    # 기사 페이지 이동
    for i in range(len(urls)):
        await page.goto(urls[i])
        await asyncio.sleep(3)
        article_text = await page.locator(f"xpath={xpath}").text_content()
        article_content.append(article_text)
        
    await browser.close()

In [50]:
article_content

['\n                    Tesla Inc. (NASDAQ: TSLA) saw its car sales drop sharply in France, Denmark, and Sweden in July 2025, extending a seven-month decline across key European markets. According to official industry data, Tesla registered 1,307 new vehicles in France last month, marking a 27% decrease compared to July 2024. The broader French auto market also faced an 8% decline in total car sales during the same period. Denmark posted an even steeper drop, with Tesla sales plunging 52% year-on-year to 336 vehicles. Registrations of the Model Y, the company’s most popular model in Europe, fell 49% in Denmark. In contrast, Denmark’s overall car sales rose 20% in July, highlighting Tesla’s underperformance in the region. The downturn was most severe in Sweden, where Tesla’s new car registrations tumbled 85.8% to just 163 vehicles, based on data from Mobility Sweden. These figures reflect broader headwinds facing Tesla in Europe. The company’s sales have dropped by more than one-third a

#### strip 함수를 써서 불필요한 공백 제거 (\n과 같은)

In [51]:
article_content = []
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    xpath = '//*[@id="view"]/div[2]/div[3]/article'
    # 기사 페이지 이동
    for i in range(len(urls)):
        await page.goto(urls[i])
        await asyncio.sleep(3)
        article_text = await page.locator(f"xpath={xpath}").text_content()
        article_text = article_text.strip() # 여기서 썼습니다.
        article_content.append(article_text)
        
    await browser.close()

In [52]:
article_content

['Tesla Inc. (NASDAQ: TSLA) saw its car sales drop sharply in France, Denmark, and Sweden in July 2025, extending a seven-month decline across key European markets. According to official industry data, Tesla registered 1,307 new vehicles in France last month, marking a 27% decrease compared to July 2024. The broader French auto market also faced an 8% decline in total car sales during the same period. Denmark posted an even steeper drop, with Tesla sales plunging 52% year-on-year to 336 vehicles. Registrations of the Model Y, the company’s most popular model in Europe, fell 49% in Denmark. In contrast, Denmark’s overall car sales rose 20% in July, highlighting Tesla’s underperformance in the region. The downturn was most severe in Sweden, where Tesla’s new car registrations tumbled 85.8% to just 163 vehicles, based on data from Mobility Sweden. These figures reflect broader headwinds facing Tesla in Europe. The company’s sales have dropped by more than one-third across the continent du

In [54]:
tesla_df1 = tesla_df[:10]

In [57]:
tesla_df1['article_content'] =article_content

C:\Users\hyukj\AppData\Local\Temp\ipykernel_22948\3546996834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla_df1['article_content'] =article_content


In [58]:
tesla_df1.head()

,index,title,url,crawled_at,article_content
0,1,Tesla Ordered to Pay $243M in Florida Autopilo...,https://econotimes.com/Tesla-Ordered-to-Pay-24...,2025-09-08 19:16:50,Tesla Inc. (NASDAQ: TSLA) saw its car sales dr...
1,2,"Tesla Car Sales in France, Denmark, and Sweden...",https://econotimes.com/Tesla-Car-Sales-in-Fran...,2025-09-08 19:16:50,Tesla Inc (NASDAQ: TSLA) has signed a $4.3 bil...
2,3,Tesla Secures $4.3B Battery Deal with LG Energ...,https://econotimes.com/Tesla-Secures-43B-Batte...,2025-09-08 19:16:50,Shares of Samsung Electronics (KS:005930) fell...
3,4,Samsung Shares Dip After Rally on $16.5B Tesla...,https://econotimes.com/Samsung-Shares-Dip-Afte...,2025-09-08 19:16:50,Tesla (NASDAQ:TSLA) CEO Elon Musk announced a ...
4,5,Tesla Inks $16.5B Chip Deal with Samsung for N...,https://econotimes.com/Tesla-Inks-165B-Chip-De...,2025-09-08 19:16:50,Tesla (NASDAQ:TSLA) CEO Elon Musk confirmed a ...


# 아주대학교 '학사' 공지사항 WebScrapping 실습

In [ ]:
https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=0

### '번호'에 공지라고 표시된 것을 제외하고, 12페이지까지 내용 추출해보기 (제목과 링크)

In [ ]:
제목의 XPath 파악
//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[12]/td[3]/div/a
//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[13]/td[3]/div/a
//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[21]/td[3]/div/a

## 첫 페이지에서 제목, 링크 추출하는 코드
- 브라우저를 띄운다. p.chromium.launch(headless=True)
- 새로운 창을 띄운다. browser.new_page()

위 두가지 규칙으로 시작

In [1]:
xpath = []
for i in range(12,22):
    k = f'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[{i}]/td[3]/div/a'
    xpath.append(k)

In [62]:
xpath

['//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[12]/td[3]/div/a',
 '//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[13]/td[3]/div/a',
 '//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[14]/td[3]/div/a',
 '//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[15]/td[3]/div/a',
 '//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[16]/td[3]/div/a',
 '//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[17]/td[3]/div/a',
 '//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[18]/td[3]/div/a',
 '//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[19]/td[3]/div/a',
 '//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[20]/td[3]/div/a',
 '//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[21]/td[3]/div/a']

In [64]:
f"xpath={xpath}"

'xpath=[\'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[12]/td[3]/div/a\', \'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[13]/td[3]/div/a\', \'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[14]/td[3]/div/a\', \'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[15]/td[3]/div/a\', \'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[16]/td[3]/div/a\', \'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[17]/td[3]/div/a\', \'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[18]/td[3]/div/a\', \'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[19]/td[3]/div/a\', \'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[20]/td[3]/div/a\', \'//*[@id="cms-content"]/div/div/div[3]/table/tbody/tr[21]/td[3]/div/a\']'

In [71]:
# 제목 추출
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()

    await page.goto("https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=0")
    await asyncio.sleep(1.5)

    for path in xpath:
        element = page.locator(f"xpath={path}")
        title = await element.text_content()
        title = title.strip()
        
        print(title)

[국제학부]국제무역현장실무 수업특강안내 -"취직도 전략이다"
(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(2025.09.09.(화) 개강)
[다산학부대학] 2025학년도 2학기 <영어2> 증원 안내
[다산학부대학] 교양 및 영역별교양 정원 외 수강신청 안내
[다산학부대학] 2025-2 교양과목 폐강 및 정원증원 안내
[재공지] [ 2025-2 유학생 수학 튜터링 International Student Math Tutoring Tutee 모집 Recruitment ]
인공지능융합학과 12기 1차 학생모집[복수/부전공]
[다산학부대학] 2025-2 교양과목 정원증원 안내
[글로벌교양학부] 25.2학기 동아리 소개를 진행할 동아리 모집합니다 (~ 25.09.02(화))
[대학원] 2025-2학기 반도체 관련 신설 과목 안내(2025.09.10.(수) 개강)


In [80]:
# 제목 + 링크 추출
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()

    await page.goto("https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=0")
    await asyncio.sleep(1.5)

    for path in xpath:
        element = page.locator(f"xpath={path}")
        title = await element.text_content()
        href = await element.get_attribute('href')
        url = f"https://www.ajou.ac.kr/kr/ajou/notice.do{href}"
        title = title.strip()
        
        
        print("제목 :", title)
        print("href :", href)
        print("url :", url)


제목 : [국제학부]국제무역현장실무 수업특강안내 -"취직도 전략이다"
href : ?mode=view&articleNo=354451&article.offset=0&articleLimit=10&srCategoryId=1
url : https://www.ajou.ac.kr/kr/ajou/notice.do?mode=view&articleNo=354451&article.offset=0&articleLimit=10&srCategoryId=1
제목 : (금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(2025.09.09.(화) 개강)
href : ?mode=view&articleNo=353635&article.offset=0&articleLimit=10&srCategoryId=1
url : https://www.ajou.ac.kr/kr/ajou/notice.do?mode=view&articleNo=353635&article.offset=0&articleLimit=10&srCategoryId=1
제목 : [다산학부대학] 2025학년도 2학기 <영어2> 증원 안내
href : ?mode=view&articleNo=354375&article.offset=0&articleLimit=10&srCategoryId=1
url : https://www.ajou.ac.kr/kr/ajou/notice.do?mode=view&articleNo=354375&article.offset=0&articleLimit=10&srCategoryId=1
제목 : [다산학부대학] 교양 및 영역별교양 정원 외 수강신청 안내
href : ?mode=view&articleNo=354208&article.offset=0&articleLimit=10&srCategoryId=1
url : https://www.ajou.ac.kr/kr/ajou/notice.do?mode=view&articleNo=354208&article.offset=0&articleLimit=10&srCategory

#### href를 통해 실제 절대 경로 찾기 (실제 웹주소)
- href는 상대경로만을 지정해줄 때가 많음.
- 기본 URL에 상대경로를 합친 방식으로 절대 경로를 지정해주어야 함.
- 이번 아주대학교 페이지의 경우, 웹주소 양식에서 "https://www.ajou.ac.kr/kr/ajou/notice.do" 까지만이 경로를 나타내며 그 이후 ? 부분은 쿼리 스트링임.

#### 다른 방식
- 개발자도구에서 href를 찾아서 실제 url에 접속해본다.
- 상대경로와 실제 웹주소를 비교하여, 규칙을 찾아낸다

```
href : ?mode=view&articleNo=354451&article.offset=0&articleLimit=10&srCategoryId=1
실제 웹 : https://www.ajou.ac.kr/kr/ajou/notice.do?mode=view&articleNo=354451&article.offset=0&articleLimit=10&srCategoryId=1
아, 실제 url을 정의해줄 때는 앞에 https://www.ajou.ac.kr/kr/ajou/notice.do 까지는 작성해주고 상대경로를 붙이면 되구나!
```

#### Playwright + Beautifulsoup 활용 본문 내용 스크래핑
- Playwright를 활용해 본문의 html 내용 전체 추출
- beautifulsoup를 활용해 html 요소 정제
- markdownify를 활용해 텍스트만을 추출

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install markdownify

In [13]:
import asyncio
from playwright.async_api import async_playwright

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()

    await page.goto("https://www.ajou.ac.kr/kr/ajou/notice.do?mode=view&articleNo=353634&article.offset=0&articleLimit=10&srCategoryId=1")
    locator = page.locator('#cms-content > div > div > div.bn-view-common01.type01 > div.b-main-box > div.b-content-box')
    # Use inner_html() to get the HTML content of that element
    html_content = await locator.inner_html()

In [3]:
html_content

'\n\n\t\t\n\t\t\t<div class="fr-view"><p style="box-sizing: border-box; margin: 0px; padding: 0px; border: 0px; font-family: &quot;Noto Sans KR&quot;, Pretendard, sans-serif; font-weight: 400; font-size: 16px; color: rgb(0, 0, 0); line-height: 25px; font-style: normal; font-variant-ligatures: normal; font-variant-caps: normal; letter-spacing: normal; orphans: 2; text-indent: 0px; text-transform: none; widows: 2; word-spacing: 0px; -webkit-text-stroke-width: 0px; white-space: normal; text-decoration-thickness: initial; text-decoration-style: initial; text-decoration-color: initial; background-color: rgb(255, 255, 255); text-align: center;"><span style="box-sizing: border-box; margin: 0px; padding: 0px; border: 0px; font-size: 30px;"><strong style="box-sizing: border-box; margin: 0px; padding: 0px; font-weight: 700;">💡 “비전공자도 환영! S전자 연구원에게 직접 배우는 반도체 실무”&nbsp;</strong></span></p><p style="box-sizing: border-box; margin: 0px; padding: 0px; border: 0px; font-family: &quot;Noto Sans KR&quot

In [14]:
from bs4 import BeautifulSoup, Comment
import requests

def remove_all_attributes(soup):
    for tag in soup.find_all(True):
        tag.attrs = {}
    return soup

def remove_useless_tags(soup):
    # 불필요한 태그 제거
    for tag in soup(['script', 'style', 'header', 'footer', 'nav']):
        tag.decompose()
    
    # 주석 제거
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()
    
    return soup

def clean_html(html_content):
    # HTML 파싱
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 모든 속성 제거
    cleaned_soup = remove_all_attributes(soup)
    cleaned_soup = remove_useless_tags(cleaned_soup)
    
    # 정제된 HTML 반환
    return str(cleaned_soup)

def clean_html_from_url(url):
    try:
        # 웹 페이지 가져오기
        response = requests.get(url)
        response.raise_for_status()  # HTTP 오류 체크
        
        # HTML 정제
        cleaned_html = clean_html(response.text)
        return cleaned_html
    
    except requests.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

cleaned_local_html = clean_html(html_content)

In [6]:
clean_html

'\n<div><p><span><strong>💡 “비전공자도 환영! S전자 연구원에게 직접 배우는 반도체 실무”\xa0</strong></span></p><p><br/></p><p><strong><span>대학원「비전공자를 위한 반도체 실무 개론」 과목 신설 안내</span></strong></p><p><strong><span>(2025.09.10.(수) 개강)</span></strong></p><p><br/></p><p>🔎 반도체는 ‘디지털 산업의 쌀’이라 불리며, 한국 경제를 이끄는 핵심 산업입니다.</p><p><br/></p><p>😮 하지만 방대한 공정과 전문적인 용어들 때문에 비전공자에게는 다소 낯설고 어렵게만 느껴집니다.</p><p><br/></p><p><br/></p><p>👉 그렇다면 <strong>2025학년도 2학기 대학원에서 개설되는 「비전공자를 위한 반도체 실무 개론」 과목</strong>을 주목해 보세요.</p><p><br/></p><p>「비전공자를 위한 반도체 실무 개론」 \xa0과목은 S전자 출신 산학협력 교수진이 직접 강의하며,\xa0</p><p>반도체 산업의 전반적인 흐름부터 최신 기술 트렌드까지 현장 실무 중심으로 배울 수 있도록 구성되어 있습니다.</p><p><br/></p><p><br/></p><ul><li><p>반도체 산업의 과거·현재·미래와 거대한 산업 생태계 구조 이해</p></li></ul><p><br/></p><ul><li><p>FAB 구조, 포토·식각·박막·CMP·세정 등 <strong>전공정 핵심 기술</strong> 학습</p></li></ul><p><br/></p><ul><li><p>패키징·HBM·테스트·신뢰성 평가 등 <strong>후공정 최신 동향 습득</strong></p></li></ul><p><br/></p><ul><li><p>반도체 제조사의 조직 구조와 직무 이해, <strong>산업 내 전략적 커뮤니케이션 역량 강화</strong></p></li></ul><p><br/></p><ul><li><p><s

In [8]:
from markdownify import markdownify as md
md(clean_html)

'**💡 “비전공자도 환영! S전자 연구원에게 직접 배우는 반도체 실무”**\n\n**대학원「비전공자를 위한 반도체 실무 개론」 과목 신설 안내**\n\n**(2025.09.10.(수) 개강)**\n\n🔎 반도체는 ‘디지털 산업의 쌀’이라 불리며, 한국 경제를 이끄는 핵심 산업입니다.\n\n😮 하지만 방대한 공정과 전문적인 용어들 때문에 비전공자에게는 다소 낯설고 어렵게만 느껴집니다.\n\n👉 그렇다면 **2025학년도 2학기 대학원에서 개설되는 「비전공자를 위한 반도체 실무 개론」 과목**을 주목해 보세요.\n\n「비전공자를 위한 반도체 실무 개론」 \xa0과목은 S전자 출신 산학협력 교수진이 직접 강의하며,\n\n반도체 산업의 전반적인 흐름부터 최신 기술 트렌드까지 현장 실무 중심으로 배울 수 있도록 구성되어 있습니다.\n\n* 반도체 산업의 과거·현재·미래와 거대한 산업 생태계 구조 이해\n\n* FAB 구조, 포토·식각·박막·CMP·세정 등 **전공정 핵심 기술** 학습\n\n* 패키징·HBM·테스트·신뢰성 평가 등 **후공정 최신 동향 습득**\n\n* 반도체 제조사의 조직 구조와 직무 이해, **산업 내 전략적 커뮤니케이션 역량 강화**\n\n* **현업 연구원 출신 교수진**이 전하는 살아 있는 실무 경험과 인사이트\n\n* **전공과 상관없이** 누구나 반도체 산업에 대한 적응력과 실무 감각 강화\n\n**📌 과목 정보**\n\n* 교과목명:**「비전공자를 위한 반도체 실무 개론」**\n\n* 담당교원: **지능형반도체공학과 한주철 산학협력교수(現 S전자 Foundry 공정기술(CMP))**\n\n* 학점/시간: **3학점/3시간**\n\n* 학수구분: **전공**\n\n* 수강번호:**O1604**\n* 수강대상: **반도체 실무 지식 습득에 관심이 있는****대학원생 및 학부 3·4학년**\n\n* 수업방식 및 평가방법: **대면 수업, 중간/기말 평가**\n\n* 수업시간: **수요일 14:00 ~ 17:00**\n\n* 강의실:\xa0**

In [15]:
# 제목, 링크, 본문 html 추출 --> df로 저장
import asyncio
from playwright.async_api import async_playwright


ajou_data = []

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()

    await page.goto("https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=0")
    await asyncio.sleep(1.5)

    for path in xpath:
        element = page.locator(f"xpath={path}")
        title = await element.text_content()
        href = await element.get_attribute('href')
        url = f"https://www.ajou.ac.kr/kr/ajou/notice.do{href}"
        title = title.strip()
        html_content = clean_html_from_url(url)
        ajou_data.append({
        'title' : title,
        'url' : url,
        'html' : html_content
        })


In [16]:
import pandas as pd
df = pd.DataFrame(ajou_data)
df

,title,url,html
0,"[국제학부]국제무역현장실무 수업특강안내 -""취직도 전략이다""",https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[국...
1,(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(...,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>(금...
2,[다산학부대학] 2025학년도 2학기 <영어2> 증원 안내,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[다...
3,[다산학부대학] 교양 및 영역별교양 정원 외 수강신청 안내,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[다...
4,[다산학부대학] 2025-2 교양과목 폐강 및 정원증원 안내,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[다...
5,[재공지] [ 2025-2 유학생 수학 튜터링 International Studen...,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[재...
6,인공지능융합학과 12기 1차 학생모집[복수/부전공],https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>인공...
7,[다산학부대학] 2025-2 교양과목 정원증원 안내,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[다...
8,[글로벌교양학부] 25.2학기 동아리 소개를 진행할 동아리 모집합니다 (~ 25.0...,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[글...
9,[대학원] 2025-2학기 반도체 관련 신설 과목 안내(2025.09.10.(수) 개강),https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,\n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[대...


In [18]:
df['html'] = df.apply(lambda row: md(row['html']), axis=1)
df.head(3)

,title,url,html
0,"[국제학부]국제무역현장실무 수업특강안내 -""취직도 전략이다""",https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,"[국제학부]국제무역현장실무 수업특강안내 -""취직도 전략이다"" | 아주대학교\n\n\..."
1,(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(...,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(...
2,[다산학부대학] 2025학년도 2학기 <영어2> 증원 안내,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,[다산학부대학] 2025학년도 2학기 <영어2> 증원 안내 | 아주대학교\n\n\n...


In [21]:
for i in range(3):
    print(f"{i+1} 번째")
    print(df['html'][i])

1 번째
[국제학부]국제무역현장실무 수업특강안내 -"취직도 전략이다" | 아주대학교



* 본문 바로가기
* 주메뉴 바로가기
* 서브메뉴 바로가기

아주광장
----

HOME

* 아주광장
  + 대학정보
  + 입학
  + 교육
  + 연구/산학
  + 학사지원
  + 대학생활
  + 아주광장
* 공지사항
  + 공지사항
  + 캘린더
  + 미디어센터
  + 커뮤니티
* 일반공지
  + 일반공지
  + 장학공지

### 일반공지

* 일반공지
* 장학공지

일반공지


일반공지

장학공지

[학사]
[국제학부]국제무역현장실무 수업특강안내 -"취직도 전략이다"

* 국제학부
* 김희주
* 작성일
  2025-09-08
* 조회수
  193

![]()

이전글

(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(2025.09.09.(화) 개강)

다음글

다음글이 없습니다.

* 목록
2 번째
(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(2025.09.09.(화) 개강) | 아주대학교



* 본문 바로가기
* 주메뉴 바로가기
* 서브메뉴 바로가기

아주광장
----

HOME

* 아주광장
  + 대학정보
  + 입학
  + 교육
  + 연구/산학
  + 학사지원
  + 대학생활
  + 아주광장
* 공지사항
  + 공지사항
  + 캘린더
  + 미디어센터
  + 커뮤니티
* 일반공지
  + 일반공지
  + 장학공지

### 일반공지

* 일반공지
* 장학공지

일반공지


일반공지

장학공지

[학사]
(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(2025.09.09.(화) 개강)

* 대학원교학팀
* 이동훈
* 작성일
  2025-09-05
* 조회수
  4449

**🚀 나만의 아이디어로 스타트업에 도전해볼까?**

**대학원「실험실 창업의 이해」 과목 신설 안내**

**(2025.09.09.(화) 개강)**

💡 창업을 한번 경험해 보고 싶은

In [ ]:
# 이제 12페이지까지 추출하는 코드를 짜보겠습니다.
# 앞서 정의한 코드를 토대로
# 1 페이지의 웹주소 https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=0
# 2 페이지의 웹주소 https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=10

# 페이지가 늘어날 때마다 웹주소의 마지막 부분인 offset가 10씩 증가합니다.
# 이 규칙에 따라 url을 변경해가며 스크래핑 시작

In [3]:
urls=[]
for i in range(0,120,10):
    url = f"https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset={i}"
    urls.append(url)
urls

['https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=0',
 'https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=10',
 'https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=20',
 'https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=30',
 'https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=40',
 'https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=50',
 'https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=60',
 'https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=70',
 'https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&articleLimit=10&srCategoryId=1&article.offset=80',
 'https://www.ajou.ac.kr/kr/ajou/notice.do?mode=list&&ar

In [6]:
import asyncio
from playwright.async_api import async_playwright

ajou_data = []

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()

    for url in urls :
        await page.goto(url)
        await asyncio.sleep(1.5)
        
        for path in xpath:
            element = page.locator(f"xpath={path}")
            title = await element.text_content()
            href = await element.get_attribute('href')
            url = f"https://www.ajou.ac.kr/kr/ajou/notice.do{href}"
            title = title.strip()
            html_content = clean_html_from_url(url)
            ajou_data.append({
                'title' : title,
                'url' : url,
                'html' : html_content
            })


        
    await browser.close()

In [7]:
import pandas as pd
df = pd.DataFrame(ajou_data)
print(df.head(3))
print(len(df))

                                               title  \
0                  [국제학부]국제무역현장실무 수업특강안내 -"취직도 전략이다"   
1  (금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(...   
2                   [다산학부대학] 2025학년도 2학기 <영어2> 증원 안내   

                                                 url  \
0  https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...   
1  https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...   
2  https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...   

                                                html  
0  \n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[국...  
1  \n<!DOCTYPE html>\n\n<html>\n<head>\n<title>(금...  
2  \n<!DOCTYPE html>\n\n<html>\n<head>\n<title>[다...  
120


In [8]:
from markdownify import markdownify as md
df['html'] = df.apply(lambda row: md(row['html']), axis=1)
df.head(3)

,title,url,html
0,"[국제학부]국제무역현장실무 수업특강안내 -""취직도 전략이다""",https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,"[국제학부]국제무역현장실무 수업특강안내 -""취직도 전략이다"" | 아주대학교\n\n\..."
1,(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(...,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(...
2,[다산학부대학] 2025학년도 2학기 <영어2> 증원 안내,https://www.ajou.ac.kr/kr/ajou/notice.do?mode=...,[다산학부대학] 2025학년도 2학기 <영어2> 증원 안내 | 아주대학교\n\n\n...


In [12]:
for i in range(5):
    print(f'{i+1} 번째')
    print(df['html'][i])
    print('==='*40)

1 번째
[국제학부]국제무역현장실무 수업특강안내 -"취직도 전략이다" | 아주대학교



* 본문 바로가기
* 주메뉴 바로가기
* 서브메뉴 바로가기

아주광장
----

HOME

* 아주광장
  + 대학정보
  + 입학
  + 교육
  + 연구/산학
  + 학사지원
  + 대학생활
  + 아주광장
* 공지사항
  + 공지사항
  + 캘린더
  + 미디어센터
  + 커뮤니티
* 일반공지
  + 일반공지
  + 장학공지

### 일반공지

* 일반공지
* 장학공지

일반공지


일반공지

장학공지

[학사]
[국제학부]국제무역현장실무 수업특강안내 -"취직도 전략이다"

* 국제학부
* 김희주
* 작성일
  2025-09-08
* 조회수
  196

![]()

이전글

(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(2025.09.09.(화) 개강)

다음글

다음글이 없습니다.

* 목록
2 번째
(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(2025.09.09.(화) 개강) | 아주대학교



* 본문 바로가기
* 주메뉴 바로가기
* 서브메뉴 바로가기

아주광장
----

HOME

* 아주광장
  + 대학정보
  + 입학
  + 교육
  + 연구/산학
  + 학사지원
  + 대학생활
  + 아주광장
* 공지사항
  + 공지사항
  + 캘린더
  + 미디어센터
  + 커뮤니티
* 일반공지
  + 일반공지
  + 장학공지

### 일반공지

* 일반공지
* 장학공지

일반공지


일반공지

장학공지

[학사]
(금일 18:00 마감예정) [대학원] 2025-2학기 창업 관련 신설 과목 안내(2025.09.09.(화) 개강)

* 대학원교학팀
* 이동훈
* 작성일
  2025-09-05
* 조회수
  4504

**🚀 나만의 아이디어로 스타트업에 도전해볼까?**

**대학원「실험실 창업의 이해」 과목 신설 안내**

**(2025.09.09.(화) 개강)**

💡 창업을 한번 경험해 보고 싶은

## 사람인 사이트에 접속해, 채용 정보를 추출 (공고 제목, 기업명)